In [0]:
!pip3 install pyLDAvis

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 59.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=8a206011abec0af46944406429fe811602f2ace21ecbd88b47109c23326de5d6
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=d7aaea63fc346fe1efa97f8cadb0d74e6d9fa6eb4d6540f073ce62a1bee6013a
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import numpy as np
import pandas as pd

# Load LDA model

In [0]:
# Initialize Parameters
lda_filename  = "data/noverbs_bigrams_islanders_fans_nt60_pa100_alphasymmetric.pickle"
print(lda_params)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


{'num_topics': 60, 'passes': 100, 'alpha': 'symmetric'}


In [0]:
lda = models.LdaModel.load(lda_filename)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
with open(lda_filename+'.pickle', 'rb') as handle:
  followers_data = pickle.load(handle)[0]
  
pyLDAvis.display(followers_data)

In [0]:
df = followers_data.topic_info
lambda_optim = 0.6
df['relevance'] = df.apply(lambda x: lambda_optim*x['logprob'] + (1-lambda_optim)*x['loglift'], axis=1)


In [0]:
import wordcloud

figure = plt.figure(figsize=(20,4*num_topics/5))
topn = 15

for i,line in enumerate(range(1,num_topics+1)):
  ax = figure.add_subplot(num_topics/5,5,i+1)
  df_ = df.loc[df['Category']==f"Topic{i+1}"].sort_values(by='relevance',ascending=False)
  scores = np.exp(df_['relevance'].values)
  scores = list(scores/sum(scores))
  words = list(df_['Term'].values)

  freqs = dict()
  for word, score in zip(words[:topn], scores[:topn]):
      freqs[word] = score
  wc = wordcloud.WordCloud(width=400,
                           height=400,
                           background_color='white',)
  wc.fit_words(freqs)
  
  ax.imshow(wc.recolor(), interpolation="bilinear",axes=ax)
  ax.set_title(f'T{i+1}',color='k',fontsize=25)
  plt.axis('off')

figure.savefig(path+f"best_topics_LDAvis_nt{lda_params['num_topics']}_pa{lda_params['passes']}_alpha{lda_params['alpha']}.png")

Output hidden; open in https://colab.research.google.com to view.